# Experimental Setup for Building Classifiers:

## Egyptian Wikipedia Dataset Filtrating:

### Quantify Bots & Humans Editors:

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

df = pd.read_csv("../Wikipedia-20240101/arzwiki-20240101/arzwiki-20240101-pages-articles-multistream-all-metadata.csv", index_col='No.')
df.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,total_bytes,total_chars,total_words,page_text,timestamp
No.,,,,,,,,,,,
1,جنوب السودان,2008-09-03,imported&gt;FadyWalker,252,58,"['Luckas-bot', 'Ghaly', 'imported&gt;FadyWalke...",10601,3826,628,علم جنوب السودان جنوب السودان هيا دولة مستقلة ...,2023-10-29T09:42:46Z
2,الاهرامات المصريه,2008-04-23,Ghaly,135,63,"['Ghaly', 'Raafat', 'GhalyBot', 'MenoBot', 'Sa...",11244,1474,249,الاهرامات الجيزه من الهضبه الجنوبيه للمجمع من ...,2023-10-01T16:49:14Z
3,اندريه اجاسى,2008-09-29,Ghaly,123,33,"['Ghaly', 'GhalyBot', 'MenoBot', 'Ramsis II', ...",13729,1197,195,اندريه كيرك اجاسى هو لاعب تنس محترف معتزل من ا...,2023-11-24T08:56:13Z
4,برودباند انترنت,2008-11-22,Ghaly,49,26,"['Ghaly', 'GhalyBot', 'Xqbot', 'JAnDbot', 'Men...",955,384,62,برودباند إنترنت اكسيس الاتصال بالإنترنت ذات ال...,2023-01-20T21:33:37Z
5,توليدو,2008-07-05,Ghaly,118,34,"['Ghaly', 'GhalyBot', 'Luckas-bot', 'Xqbot', '...",11988,4526,750,توليدو هى مدينه بلديه فى وسط اسبانيا على بعد ك...,2023-10-22T21:14:21Z


In [2]:
print(f"Number of Total Articles = {df.shape[0]:,}")

Number of Total Articles = 736,107


In [3]:
import re, requests
from bs4 import BeautifulSoup

def get_arwikis_bots_list():
    bots = []
    
    for wiki in ['ar', 'arz', 'ary']:
        if wiki=='arz':
            url = f'https://{wiki}.wikipedia.org/wiki/خاص:عرض_اليوزرات?username=&group=bot&wpsubmit=&wpFormIdentifier=mw-listusers-form&limit=1000'
        
        else: 
            url = f'https://{wiki}.wikipedia.org/wiki/خاص:عرض_المستخدمين?username=&group=bot&wpsubmit=&wpFormIdentifier=mw-listusers-form&limit=1000'
    
        bots_list = re.findall("<" + "bdi" + ">(.*?)</" + "bdi" + ">", str(BeautifulSoup(requests.get(url).content, "html.parser")))
        bots = bots + bots_list

    return list(set(bots)) 


arwikis_bots_list = get_arwikis_bots_list()
print(f"## Found {len(arwikis_bots_list)} Bots in {{AR, ARZ, ARY}} WIKIs")

## Found 54 Bots in {AR, ARZ, ARY} WIKIs


In [4]:
def quantify_bots_humans_edits(top_editors, arwikis_bots_list):    
    n_bots = 0
    
    top_editors = top_editors.replace("[","").replace("]","").replace("'","").replace(" ", "").split(",")

    for editor in top_editors:
        if editor in arwikis_bots_list or 'Bot' in  editor: n_bots += 1
    
    return (round(n_bots/len(top_editors)*100, 2)), (100-(round(n_bots/len(top_editors)*100, 2)))


df['bots_editors_%'] = df['top_editors'].apply(lambda top_editors: quantify_bots_humans_edits(top_editors, arwikis_bots_list)[0])
df['humans_editors_%'] = df['top_editors'].apply(lambda top_editors: quantify_bots_humans_edits(top_editors, arwikis_bots_list)[1])

df = df[['page_title', 'creation_date', 'creator_name', 'total_edits', 'total_editors', 'top_editors', 'bots_editors_%', 'humans_editors_%', 'total_bytes', 'total_chars', 'total_words', 'page_text']]
df.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words,page_text
No.,,,,,,,,,,,,
1,جنوب السودان,2008-09-03,imported&gt;FadyWalker,252,58,"['Luckas-bot', 'Ghaly', 'imported&gt;FadyWalke...",39.66,60.34,10601,3826,628,علم جنوب السودان جنوب السودان هيا دولة مستقلة ...
2,الاهرامات المصريه,2008-04-23,Ghaly,135,63,"['Ghaly', 'Raafat', 'GhalyBot', 'MenoBot', 'Sa...",25.40,74.60,11244,1474,249,الاهرامات الجيزه من الهضبه الجنوبيه للمجمع من ...
3,اندريه اجاسى,2008-09-29,Ghaly,123,33,"['Ghaly', 'GhalyBot', 'MenoBot', 'Ramsis II', ...",33.33,66.67,13729,1197,195,اندريه كيرك اجاسى هو لاعب تنس محترف معتزل من ا...
4,برودباند انترنت,2008-11-22,Ghaly,49,26,"['Ghaly', 'GhalyBot', 'Xqbot', 'JAnDbot', 'Men...",30.77,69.23,955,384,62,برودباند إنترنت اكسيس الاتصال بالإنترنت ذات ال...
5,توليدو,2008-07-05,Ghaly,118,34,"['Ghaly', 'GhalyBot', 'Luckas-bot', 'Xqbot', '...",41.18,58.82,11988,4526,750,توليدو هى مدينه بلديه فى وسط اسبانيا على بعد ك...


In [5]:
df.describe()

,total_edits,total_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words
count,736107.000000,736107.000000,736107.000000,736107.000000,7.361070e+05,736107.000000,736107.000000
mean,6.084625,3.331005,61.909479,38.090521,2.072879e+03,610.544206,100.900059
std,92.441210,6.589833,14.087735,14.087735,4.039134e+03,1250.328204,217.818069
min,1.000000,1.000000,0.000000,16.670000,5.150000e+02,233.000000,50.000000
25%,3.000000,2.000000,50.000000,25.000000,9.750000e+02,338.000000,57.000000
50%,5.000000,3.000000,66.670000,33.330000,1.165000e+03,413.000000,69.000000
75%,7.000000,4.000000,75.000000,50.000000,3.648000e+03,919.000000,149.000000
max,78927.000000,5256.000000,83.330000,100.000000,1.217036e+06,399641.000000,74009.000000


### Apply Heuristic Rules for *Before* Translation:

In [6]:
1# articles created before '2019-12-31'   
2# articles with greater than 5 edits   
3# articles with greater than 3 editors   
4# articles with greater than 50% human editors

before_translation = df[(df['creation_date'] <= '2019-12-31') & \
                    (df['total_edits'] > 5) & (df['total_editors'] > 3) & \
                    (df['humans_editors_%'] > 50)].sort_values(by='creation_date')

before_translation.to_csv('arzwiki-20240101-before-translation.csv', index=False)

before_translation.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words,page_text
No.,,,,,,,,,,,,
8959,الصفحه الرئيسيه,2008-04-02,Ghaly,477,24,"['Ghaly', 'Raafat', 'HitomiAkane', 'Eskandaran...",0.00,100.00,17089,2046,334,بك مشروع موسوعه حره اى حد ممكن يساهم فى كتابته...
2667,اللغه المصريه الحديثه,2008-04-03,Ghaly,643,88,"['MasriDefend', 'Ghaly', 'Esperfulmo', 'Samsam...",18.18,81.82,68468,26738,4868,اللغه المصريه الحديثه زى ما سماها بيومى قنديل ...
209,مصريين,2008-04-04,Ghaly,327,57,"['Samsam22', 'Ghaly', 'Ghofran Ali Abdalalim',...",22.81,77.19,157504,65661,11163,المصريين اللغه المصرى القديمه رمت كمت rmṯ km ا...
233,ويكيبيديا,2008-04-04,Ghaly,237,65,"['Ghaly', 'Xqbot', 'GhalyBot', 'SieBot', 'Espe...",32.31,67.69,79610,5439,879,ويكيپيديا wikipedia موسوعه مفتوحه متعدده اللغا...
207,مصر,2008-04-05,Ghaly,635,129,"['Samsam22', 'Esperfulmo', 'Ramsis II', 'Mody1...",20.93,79.07,54767,20542,3526,مصر اللغه المصريه القديمه كمت km اللغه القبطيه...


In [7]:
print(f"Number of Articles Before Translation = {before_translation.shape[0]:,}")

Number of Articles Before Translation = 11,126


### Apply Heuristic Rules for *After* Translation:

In [8]:
1# articles created between '2019-12-01 and '2023-12-01'    
2# Discard new young articles (age less than 30 days) from the dump files dates 2024-01-01 
3# articles with less than 5 edits    
4# articles with less than 3 editors    
5# articles with greater than or equal 50% bot editors
6# articles created by these two registered users {'HitomiAkane', 'Al-Dandoon'}

after_translation = df[(df['creation_date'] > '2019-12-01') & \
                   (df['creation_date'] < '2023-12-01') & (df['total_edits'] < 5) & (df['total_editors'] < 3) & \
                    (df['bots_editors_%'] >= 50) & ((df['creator_name'] == 'HitomiAkane') | (df['creator_name'] == 'Al-Dandoon'))].sort_values(by='creation_date') 

after_translation.to_csv('arzwiki-20240101-after-translation.csv', index=False)

after_translation.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words,page_text
No.,,,,,,,,,,,,
16175,عبد الحفيظ كردر,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1408,362,59,عبد الحفيظ كردر لاعب كريكت سياسى دبلوماسى من ب...
16232,جون مانرز-ساتون,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1100,331,56,جون مانرز ساتون الفيكونت كانتربيرى التالت لاعب...
16224,جونى بريجز,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1189,403,66,جونى بريجز لاعب كريكت لاعب اتحاد الرجبى من الم...
16223,ستيف هارميسون,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1150,375,59,ستيف هارميسون لاعب كريكت مدرب كوره قدم لاعب كو...
16217,فريمان فريمان توماس,2019-12-13,HitomiAkane,3,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1398,533,85,فريمان فريمان توماس مركيز ويلينجدون الاول لاعب...


In [9]:
print(f"Number of Articles After Translation = {after_translation.shape[0]:,}")

Number of Articles After Translation = 155,275


## Egyptian Wikipedia Dataset Labeling & Cleaning:

### Cleaning Procedure:

In [10]:
import re

def clean_page_text(text):
    text = re.sub(r'[^\w\s]', ' ', text) #Replaces the non-alphanumeric characters with spaces.
    text = re.sub(r'[^\u0600-\u06FF]', ' ', text) #Replaces the non-Arabic characters with spaces.
    text = re.sub(r'\s+', ' ', text) #Replaces extra spaces with a single space.
    return text

### Apply Labeling & Cleaning on *Before* Translation:

In [11]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

before_translation = pd.read_csv("arzwiki-20240101-before-translation.csv")
before_translation.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words,page_text
0,الصفحه الرئيسيه,2008-04-02,Ghaly,477,24,"['Ghaly', 'Raafat', 'HitomiAkane', 'Eskandaran...",0.00,100.00,17089,2046,334,بك مشروع موسوعه حره اى حد ممكن يساهم فى كتابته...
1,اللغه المصريه الحديثه,2008-04-03,Ghaly,643,88,"['MasriDefend', 'Ghaly', 'Esperfulmo', 'Samsam...",18.18,81.82,68468,26738,4868,اللغه المصريه الحديثه زى ما سماها بيومى قنديل ...
2,مصريين,2008-04-04,Ghaly,327,57,"['Samsam22', 'Ghaly', 'Ghofran Ali Abdalalim',...",22.81,77.19,157504,65661,11163,المصريين اللغه المصرى القديمه رمت كمت rmṯ km ا...
3,ويكيبيديا,2008-04-04,Ghaly,237,65,"['Ghaly', 'Xqbot', 'GhalyBot', 'SieBot', 'Espe...",32.31,67.69,79610,5439,879,ويكيپيديا wikipedia موسوعه مفتوحه متعدده اللغا...
4,مصر,2008-04-05,Ghaly,635,129,"['Samsam22', 'Esperfulmo', 'Ramsis II', 'Mody1...",20.93,79.07,54767,20542,3526,مصر اللغه المصريه القديمه كمت km اللغه القبطيه...


In [12]:
before_translation = before_translation.drop(columns=['creation_date', 'creator_name', 'top_editors', 'bots_editors_%', 'humans_editors_%'])
before_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text
0,الصفحه الرئيسيه,477,24,17089,2046,334,بك مشروع موسوعه حره اى حد ممكن يساهم فى كتابته...
1,اللغه المصريه الحديثه,643,88,68468,26738,4868,اللغه المصريه الحديثه زى ما سماها بيومى قنديل ...
2,مصريين,327,57,157504,65661,11163,المصريين اللغه المصرى القديمه رمت كمت rmṯ km ا...
3,ويكيبيديا,237,65,79610,5439,879,ويكيپيديا wikipedia موسوعه مفتوحه متعدده اللغا...
4,مصر,635,129,54767,20542,3526,مصر اللغه المصريه القديمه كمت km اللغه القبطيه...


In [13]:
# Apply the Cleaning Procedure
before_translation['page_text_cleaned'] = before_translation['page_text'].apply(clean_page_text)
before_translation

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned
0,الصفحه الرئيسيه,477,24,17089,2046,334,بك مشروع موسوعه حره اى حد ممكن يساهم فى كتابته...,بك مشروع موسوعه حره اى حد ممكن يساهم فى كتابته...
1,اللغه المصريه الحديثه,643,88,68468,26738,4868,اللغه المصريه الحديثه زى ما سماها بيومى قنديل ...,اللغه المصريه الحديثه زى ما سماها بيومى قنديل ...
2,مصريين,327,57,157504,65661,11163,المصريين اللغه المصرى القديمه رمت كمت rmṯ km ا...,المصريين اللغه المصرى القديمه رمت كمت اللغه ال...
3,ويكيبيديا,237,65,79610,5439,879,ويكيپيديا wikipedia موسوعه مفتوحه متعدده اللغا...,ويكيپيديا موسوعه مفتوحه متعدده اللغات المشروع ...
4,مصر,635,129,54767,20542,3526,مصر اللغه المصريه القديمه كمت km اللغه القبطيه...,مصر اللغه المصريه القديمه كمت اللغه القبطيه كي...
...,...,...,...,...,...,...,...,...
11121,تيست درايف III: ذا بايشن,9,5,1782,396,74,تيست درايڤ iii ذا پايشن انجليزى test drive iii...,تيست درايڤ ذا پايشن انجليزى ده ڤيديو لعب اللى ...
11122,حسن مختار صقر,17,5,1679,716,130,حسن مختار صقر كان مطرب شعبى ممثل من مصر حياته ...,حسن مختار صقر كان مطرب شعبى ممثل من مصر حياته ...
11123,31910 Moustafa (كويكب),12,5,1157,416,65,moustafa كويكب فى حزام الكويكبات اللى بين كوكب...,كويكب فى حزام الكويكبات اللى بين كوكب المريخ ...
11124,محمد راغب,14,8,1330,445,80,محمد راغب ممثل مصرى ابتدا مشواره الفنى فى نهاي...,محمد راغب ممثل مصرى ابتدا مشواره الفنى فى نهاي...


In [14]:
# Check and drop empty 'page_text_cleaned'
if before_translation['page_text_cleaned'].isnull().any(): 
    before_translation = before_translation.dropna(subset=['page_text_cleaned'])

In [15]:
# Make a sample of 10K articles
before_translation = before_translation.sample(10_000).reset_index(drop=True)
before_translation.shape[0]

10000

In [16]:
# Label the articles before translation as 'human-generated'
before_translation['label'] = 'human-generated'
before_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,سماره,27,9,1896,752,131,سماره اغنيه مصريه غنتها نجاة الصغيره كلماتها س...,سماره اغنيه مصريه غنتها نجاة الصغيره كلماتها س...,human-generated
1,منى ذو الفقار,46,13,19974,3879,638,منى ذو الفقار هي محامية مصرية وعضو حقوقية ومن ...,منى ذو الفقار هي محامية مصرية وعضو حقوقية ومن ...,human-generated
2,نمر تليفونات مهمه فى مصر,42,10,3242,1163,168,نص بديل نمر تليفونات مهمه فى مصر الطوارئ الاسع...,نص بديل نمر تليفونات مهمه فى مصر الطوارئ الاسع...,human-generated
3,شحاته هارون,36,11,4829,2357,428,شحاتة هارون اتولد فى يناير لأب ام مصريين اصوله...,شحاتة هارون اتولد فى يناير لأب ام مصريين اصوله...,human-generated
4,ايفيلين عشم الله,24,7,2962,767,135,ايفيلين عشم الله فنانة تشك بعديه صحفيه مصريه ح...,ايفيلين عشم الله فنانة تشك بعديه صحفيه مصريه ح...,human-generated


In [17]:
before_translation.to_csv("arzwiki-20240101-before-translation-10k.csv", index=False)

### Apply Labeling & Cleaning on *After* Translation:

In [18]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

after_translation = pd.read_csv("arzwiki-20240101-after-translation.csv")
after_translation.head()

,page_title,creation_date,creator_name,total_edits,total_editors,top_editors,bots_editors_%,humans_editors_%,total_bytes,total_chars,total_words,page_text
0,عبد الحفيظ كردر,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1408,362,59,عبد الحفيظ كردر لاعب كريكت سياسى دبلوماسى من ب...
1,جون مانرز-ساتون,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1100,331,56,جون مانرز ساتون الفيكونت كانتربيرى التالت لاعب...
2,جونى بريجز,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1189,403,66,جونى بريجز لاعب كريكت لاعب اتحاد الرجبى من الم...
3,ستيف هارميسون,2019-12-13,HitomiAkane,4,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1150,375,59,ستيف هارميسون لاعب كريكت مدرب كوره قدم لاعب كو...
4,فريمان فريمان توماس,2019-12-13,HitomiAkane,3,2,"['GhalyBot', 'HitomiAkane']",50.0,50.0,1398,533,85,فريمان فريمان توماس مركيز ويلينجدون الاول لاعب...


In [19]:
after_translation = after_translation.drop(columns=['creation_date', 'creator_name', 'top_editors', 'bots_editors_%', 'humans_editors_%'])
after_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text
0,عبد الحفيظ كردر,4,2,1408,362,59,عبد الحفيظ كردر لاعب كريكت سياسى دبلوماسى من ب...
1,جون مانرز-ساتون,4,2,1100,331,56,جون مانرز ساتون الفيكونت كانتربيرى التالت لاعب...
2,جونى بريجز,4,2,1189,403,66,جونى بريجز لاعب كريكت لاعب اتحاد الرجبى من الم...
3,ستيف هارميسون,4,2,1150,375,59,ستيف هارميسون لاعب كريكت مدرب كوره قدم لاعب كو...
4,فريمان فريمان توماس,3,2,1398,533,85,فريمان فريمان توماس مركيز ويلينجدون الاول لاعب...


In [20]:
# Apply the Cleaning Procedure
after_translation['page_text_cleaned'] = after_translation['page_text'].apply(clean_page_text)
after_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned
0,عبد الحفيظ كردر,4,2,1408,362,59,عبد الحفيظ كردر لاعب كريكت سياسى دبلوماسى من ب...,عبد الحفيظ كردر لاعب كريكت سياسى دبلوماسى من ب...
1,جون مانرز-ساتون,4,2,1100,331,56,جون مانرز ساتون الفيكونت كانتربيرى التالت لاعب...,جون مانرز ساتون الفيكونت كانتربيرى التالت لاعب...
2,جونى بريجز,4,2,1189,403,66,جونى بريجز لاعب كريكت لاعب اتحاد الرجبى من الم...,جونى بريجز لاعب كريكت لاعب اتحاد الرجبى من الم...
3,ستيف هارميسون,4,2,1150,375,59,ستيف هارميسون لاعب كريكت مدرب كوره قدم لاعب كو...,ستيف هارميسون لاعب كريكت مدرب كوره قدم لاعب كو...
4,فريمان فريمان توماس,3,2,1398,533,85,فريمان فريمان توماس مركيز ويلينجدون الاول لاعب...,فريمان فريمان توماس مركيز ويلينجدون الاول لاعب...


In [21]:
# Check and drop empty 'page_text_cleaned'
if after_translation['page_text_cleaned'].isnull().any(): 
    after_translation = after_translation.dropna(subset=['page_text_cleaned'])

In [22]:
# Make a sample of 10K articles
after_translation = after_translation.sample(10_000).reset_index(drop=True)
after_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned
0,جوديث سكوديلنى,3,2,950,330,51,جوديث سكوديلنى سياسيه من المانيا حياتها جوديث ...,جوديث سكوديلنى سياسيه من المانيا حياتها جوديث ...
1,وادى قصر الحير,2,2,1124,340,58,وادى قصر الحير هوا مجرى مائى فى سوريا المكان و...,وادى قصر الحير هوا مجرى مائى فى سوريا المكان و...
2,مايا ريهبيرج,2,2,1265,451,77,مايا ريهبيرج منافسه العاب قوى من المانيا حياته...,مايا ريهبيرج منافسه العاب قوى من المانيا حياته...
3,تيموثى ستامبس,3,2,972,302,54,تيموثى ستامبس كان سياسى من زيمبابوى ويلز حياته...,تيموثى ستامبس كان سياسى من زيمبابوى ويلز حياته...
4,جول كليفينوت,3,2,1015,353,64,جول كليفينوت كان لاعب كورة الماء سباح من فرنسا...,جول كليفينوت كان لاعب كورة الماء سباح من فرنسا...


In [23]:
# Label the articles after translation as 'template-translated'
after_translation['label'] = 'template-translated'
after_translation.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,جوديث سكوديلنى,3,2,950,330,51,جوديث سكوديلنى سياسيه من المانيا حياتها جوديث ...,جوديث سكوديلنى سياسيه من المانيا حياتها جوديث ...,template-translated
1,وادى قصر الحير,2,2,1124,340,58,وادى قصر الحير هوا مجرى مائى فى سوريا المكان و...,وادى قصر الحير هوا مجرى مائى فى سوريا المكان و...,template-translated
2,مايا ريهبيرج,2,2,1265,451,77,مايا ريهبيرج منافسه العاب قوى من المانيا حياته...,مايا ريهبيرج منافسه العاب قوى من المانيا حياته...,template-translated
3,تيموثى ستامبس,3,2,972,302,54,تيموثى ستامبس كان سياسى من زيمبابوى ويلز حياته...,تيموثى ستامبس كان سياسى من زيمبابوى ويلز حياته...,template-translated
4,جول كليفينوت,3,2,1015,353,64,جول كليفينوت كان لاعب كورة الماء سباح من فرنسا...,جول كليفينوت كان لاعب كورة الماء سباح من فرنسا...,template-translated


In [24]:
after_translation.to_csv("arzwiki-20240101-after-translation-10k.csv", index=False)

### Put all together:

In [25]:
all = pd.concat([before_translation, after_translation])

for i in range(all.shape[0]): 
    all = all.sample(frac=1).reset_index(drop=True)

all.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,هوجو تشافيز,33,12,2596,867,135,هوجو تشافيز hugo chávez كان رئيس ڤينيزويلا نمر...,هوجو تشافيز كان رئيس ڤينيزويلا نمره حياته هوجو...,human-generated
1,انتونى ويلدينج,35,8,5873,958,142,انتونى ويلدينج لاعب كريكت محامى ضابط لاعب تنس ...,انتونى ويلدينج لاعب كريكت محامى ضابط لاعب تنس ...,human-generated
2,موكارام تورجونبايڤا,2,2,1205,479,76,موكارام تورجونبايڤا كانت ممثله راقصه باليه معل...,موكارام تورجونبايڤا كانت ممثله راقصه باليه معل...,template-translated
3,القديس كليمنت الاول,23,7,2188,333,52,القديس كليمنت الاول هو البابا الرابع فى الكنيس...,القديس كليمنت الاول هو البابا الرابع فى الكنيس...,human-generated
4,30 نوفمبر,61,32,898,315,56,نوفمبر هو اليوم ال ال فى السنة الكبيسة من ايام...,نوفمبر هو اليوم ال ال فى السنة الكبيسة من ايام...,human-generated


In [26]:
all.shape

(20000, 9)

In [27]:
all.to_csv('arzwiki-20240101-all-20k.csv', index=False)